## Code Generator

Using Gemini-2.0-flash to convert python code to C++

In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# Env variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

In [13]:
openai = OpenAI()
MODEL = "gemini-2.0-flash"

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++"
system_message += "Reespond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments"
system_message += "The C++ response neeeds to produce an identical output in the fastest posssible time"

In [30]:
def user_promtp_for(python):
    user_prompt = "Rewrite this Python code in C++ witht the fastest possible implementation that prodcues identical output un the least time."
    user_prompt += "Respons only with C++ codes; do not explain your work other than a few comments"
    user_prompt += "Pay attention to tnumber types to ensure no int overflows. Remember to  #includeal the necessary C++ packages sush as iomanip. \n\n"
    user_prompt += " Check this warning: unsigned conversion from ‘long int’ to ‘unsigned int’ changes value from ‘4294967296’ to ‘0’ [-Woverflow]"
    user_prompt += python
    return user_prompt

In [31]:
def messages_for(python):
    return [
        {"role":"system", "content":system_message},
        {"role":"user", "content":user_promtp_for(python)}
    ]

In [32]:
# Write the code to optimized.cpp
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp","w") as f:
        f.write(code)

In [8]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [15]:
def optimize_gpt(python):
    stream = gemini_via_openai_client.chat.completions.create(
        model=MODEL,
        messages=messages_for(python),
        stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [16]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 31.162319 seconds


In [17]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i * param1) - param2;
        result -= (1.0 / j);
        j = static_cast<double>(i * param1) + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000LL, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    double execution_time = static_cast<double>(duration.count()) / 1000000.0;

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << execution_time << " seconds" << std::endl;

    retu

In [18]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

'clang++' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
%%script bash
g++ optimized.cpp


In [24]:
%%script bash

# Compile the C++ code
g++ optimized.cpp -o optimized

# Print a separator
#echo "----- Output -----"

# Run and time the execution
TIMEFORMAT="Execution Time: %R seconds"
time ./optimized


Result: 3.141592658589
Execution Time: 0.940712 seconds


Execution Time: 0.961 seconds


In [25]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [26]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 92.089085 seconds


In [33]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <iomanip>

// LCG generator function
unsigned int lcg(unsigned int& value, unsigned int a, unsigned int c, unsigned long long m) {
    value = (static_cast<unsigned long long>(a) * value + c) % m;
    return value;
}

// Function to calculate the maximum subarray sum
long long max_subarray_sum(int n, unsigned int seed, int min_val, int max_val) {
    std::vector<int> random_numbers(n);
    unsigned int value = seed;
    for (int i = 0; i < n; ++i) {
        value = lcg(value, 1664525, 1013904223, 4294967296ULL);
        random_numbers[i] = (value % (max_val - min_val + 1)) + min_val;
    }

    long long max_sum = std::numeric_limits<long long>::min();
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    

In [35]:
%%script bash

# Compile the C++ code
g++ optimized.cpp -o optimized

# Print a separator
#echo "----- Output -----"

# Run and time the execution
TIMEFORMAT="Execution Time: %R seconds"
time ./optimized


Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.007383 seconds


Execution Time: 0.040 seconds


In [36]:
def stream_gpt(python):
    stream = gemini_via_openai_client.chat.completions.create(
        model=MODEL,
        messages=messages_for(python),
        stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [39]:
def optimize(python,model):
    if model == "GEMINI":
        result = stream_gpt(python)
    elif model == "GPT":
        result = stream_gpt(python)
    else:
        raise ValueError("Unknown Model")
    for stream_so_far in result:
        yield stream_so_far

In [41]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ Code", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GEMINI","GPT"], label="Select Model",value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python,model], outputs=[cpp])
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [43]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [49]:
%%script bash
g++ optimized.cpp -o optimized
./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.007661 seconds


In [ ]:
def exceute_cpp(code):
    write_output(code)
    try:
        compile_cmd = 

In [50]:
def execute_cpp(code):
    write_output(code)
    try:
        # Run the same commands as in %%script bash using a single Bash shell
        bash_command = """
        g++ optimized.cpp -o optimized
        ./optimized
        """
        result = subprocess.run(["bash", "-c", bash_command], check=True, text=True, capture_output=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [54]:
css = """
.python {bacgbround-color: #306998;}
.cpp {background-color: #055;}"""

In [58]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python Code", lines=10)
        cpp = gr.Textbox(label="C++ Code",lines=10)
    with gr.Row():
        model = gr.Dropdown(["GEMINI","GPT"], label="Select the model", value="GEMINI")
    with gr.Row():
        convert = gr.Button("Convert Code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python Result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ Result", elem_classes=["cpp"])

    convert.click(optimize, inputs = [python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)